In [1]:
import pandas as pd
import numpy as np
from numpy import linalg
from numpy.linalg import cholesky
import statsmodels as sm
import statsmodels.formula.api as smf
from matplotlib import pyplot as plt
import math
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.var_model import FEVD
from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv("c://j-works22/works/data/macro_month.csv")
y=df['y']
x1=df['x1']
x2=df['x2']
x3=df['x3']
x4=df['x4']
x5=df['x5']
z=pd.concat([x1,x2,x3,x4,x5],axis=1)
n=len(y)
z=np.array(z)

# x=StandardScaler(with_mean=True, with_std=True).fit_transform(x)

In [3]:
w=np.matmul(z.T, z)   
evals, evecs = np.linalg.eig(w)
# np.shape(evecs)

# Eigenvalues and eigenvectors as descending order
sort=np.argsort(-1*evals)
vals=evals[sort]
vecs=evecs[:, sort]

# the largest eigenvalue and eigenvector
vals0=vals[0]
vals1=vals[1]
vecs0=vecs[:,0]
vecs1=vecs[:,1]

# f0 is the the largest factor
g0=np.sqrt(n)*vecs0
g1=np.sqrt(n)*vecs1
g=np.vstack((g0,g1))
g=g.T

f=np.matmul(z, g)/n

# np.shape(f)
# rescaled factor
# factor0=np.array(f0)
#f0=f0.reshape(-1,1)
#f0=StandardScaler(with_mean=True, with_std=True).fit_transform(f0)

In [4]:
f=np.array(f)
w=np.column_stack((f, y))
# w.shape[0]
# w.shape[1]

In [5]:
# VAR estimation 
w=np.column_stack((f, y))
var=VAR(w).fit(maxlags=10, ic='bic')
# compute the cholesky factorfrom VAR residuals
rsd=var.resid
sigma=np.matmul(rsd.T, rsd)/len(rsd)
pm=cholesky(sigma)
pm

array([[ 6.85062540e-02,  0.00000000e+00,  0.00000000e+00],
       [-5.61281396e-05,  5.99622873e-02,  0.00000000e+00],
       [-5.27556772e-03, -5.71873644e-03,  1.12192509e-01]])

In [6]:
fd=var.fevd(10)
fd.summary()

FEVD for y1
           y1        y2        y3
0    1.000000  0.000000  0.000000
1    0.998923  0.000912  0.000165
2    0.998195  0.001547  0.000258
3    0.997841  0.001888  0.000272
4    0.997696  0.002033  0.000271
5    0.997621  0.002084  0.000295
6    0.997549  0.002099  0.000353
7    0.997461  0.002101  0.000438
8    0.997357  0.002101  0.000541
9    0.997244  0.002101  0.000655

FEVD for y2
           y1        y2        y3
0    0.000001  0.999999  0.000000
1    0.000711  0.994895  0.004394
2    0.002081  0.992471  0.005447
3    0.002801  0.991714  0.005485
4    0.003089  0.991353  0.005558
5    0.003175  0.990954  0.005871
6    0.003192  0.990452  0.006356
7    0.003191  0.989884  0.006924
8    0.003190  0.989290  0.007520
9    0.003192  0.988695  0.008113

FEVD for y3
           y1        y2        y3
0    0.002201  0.002586  0.995214
1    0.007429  0.005930  0.986641
2    0.010337  0.008532  0.981132
3    0.012267  0.010456  0.977278
4    0.013649  0.011841  0.974509
5    0.014

In [76]:
# recursive fevd
m=len(w)
for i in range(24):
    wa=w[0:m-24+1+i,]
    vara=VAR(wa).fit(2)
    fevd=vara.fevd(12)
    print("n=", m-24+1+i)
    print(fevd.summary())

n= 235
FEVD for y1
            y1        y2        y3
0     1.000000  0.000000  0.000000
1     0.998149  0.001613  0.000238
2     0.996219  0.003319  0.000462
3     0.995189  0.004237  0.000574
4     0.994768  0.004631  0.000601
5     0.994624  0.004775  0.000601
6     0.994572  0.004822  0.000606
7     0.994540  0.004834  0.000626
8     0.994506  0.004836  0.000658
9     0.994464  0.004836  0.000700
10    0.994417  0.004836  0.000747
11    0.994368  0.004836  0.000796

FEVD for y2
            y1        y2        y3
0     0.000125  0.999875  0.000000
1     0.000117  0.993351  0.006533
2     0.000501  0.991594  0.007905
3     0.000692  0.991414  0.007894
4     0.000750  0.991063  0.008187
5     0.000759  0.990281  0.008960
6     0.000758  0.989212  0.010030
7     0.000759  0.988009  0.011233
8     0.000762  0.986763  0.012475
9     0.000767  0.985523  0.013710
10    0.000774  0.984310  0.014916
11    0.000781  0.983133  0.016085

FEVD for y3
            y1        y2        y3
0     0.00